In [1]:
import numpy as np


In [2]:
# 1) 
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = np.array([1, 2, 3])
c = 5


def func(x):
    x_transpose = np.transpose(x)
    return 1 / 2 * (x @ A @ x_transpose) - b @ x_transpose + c


def f_grad(x):
    return A @ x - b


lambdas = list(np.linalg.eig(A)[0])


In [3]:
def project(x):
    return np.array([max(min(x_i, 1), 0) for x_i in x])


def projected_grad():
    alpha = 2.0 / (sum(lambdas))
    traj_opt_step = []
    n = len(b)
    x_start = np.array([1 for _ in range(n)])
    traj_opt_step.append(x_start.copy())
    cur_x = x_start.copy()
    for i in range(200):
        cur_x = cur_x - alpha * f_grad(cur_x)
        cur_x = project(cur_x)
        traj_opt_step.append(cur_x.copy())
    return traj_opt_step


print(projected_grad())


[array([1, 1, 1]), array([0.33333333, 0.        , 0.        ]), array([0.42222222, 0.08888889, 0.08888889]), array([0.44, 0.  , 0.  ]), array([0.51466667, 0.032     , 0.        ]), array([0.57084444, 0.00284444, 0.        ]), array([0.62730667, 0.        , 0.        ]), array([0.67699911, 0.        , 0.        ]), array([0.7200659, 0.       , 0.       ]), array([0.75739044, 0.        , 0.        ]), array([0.78973838, 0.        , 0.        ]), array([0.81777327, 0.        , 0.        ]), array([0.84207016, 0.        , 0.        ]), array([0.86312748, 0.        , 0.        ]), array([0.88137715, 0.        , 0.        ]), array([0.89719353, 0.        , 0.        ]), array([0.91090106, 0.        , 0.        ]), array([0.92278092, 0.        , 0.        ]), array([0.93307679, 0.        , 0.        ]), array([0.94199989, 0.        , 0.        ]), array([0.94973324, 0.        , 0.        ]), array([0.95643547, 0.        , 0.        ]), array([0.96224407, 0.        , 0.        ]), array([0.967

In [4]:
# 2) 1
from scipy.optimize import linprog


def get_interior_point(A, b):
    n = len(A[0])
    bounds_for_coordinates = [(None, None) for _ in range(n)]
    coefficients_x_i = [0 for _ in range(n)]

    def get_x_i_bounds(i):
        bounds = None
        coefficients_x_i[i] = 1
        min_bound = linprog(coefficients_x_i,
                            A_ub=A,
                            b_ub=b,
                            bounds=bounds_for_coordinates)
        coefficients_x_i[i] = -1
        max_bound = linprog(coefficients_x_i,
                            A_ub=A,
                            b_ub=b,
                            bounds=bounds_for_coordinates)
        coefficients_x_i[i] = 0
        min_x_i = min_bound.fun
        max_x_i = -max_bound.fun
        if min_bound.status == 0 and max_bound.status == 0:
            bounds = [min_x_i, max_x_i]
        elif min_bound.status == 0:
            bounds = [min_x_i, min_x_i + 2]
        elif max_bound.status == 0:
            bounds = [max_x_i - 2, max_x_i]
        elif min_bound.status == 3 and max_bound.status == 3:
            bounds = [-8, 8]
        return bounds

    inner_point = []
    for i in range(n):
        bounds = get_x_i_bounds(i)
        if bounds is None or bounds[0] == bounds[1]:
            return None
        bounds_for_coordinates[i] = (bounds[0], bounds[1])
        inner_point.append((bounds[0] + bounds[1])/2)
    return inner_point


In [5]:
# 2) 2
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = np.array([1, 2, 3])
c = 5
EqA = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [-1, 0, 0, ], [0, -1, 0], [0, 0, -1]])
EqB = np.array([[1], [1], [1], [0], [0], [0]])
m = len(EqB)


def grad_Fi(x):
    grad = 0
    for i in range(m):
        grad += 1.0 / (EqB[i] - EqA[i] @ x) * EqA[i]
    return np.transpose(grad)


def hessian_Fi(x):
    d = np.diag(np.array([1.0 / (EqB[i][0] - EqA[i] @ x) ** 2 for i in range(m)]))
    return EqA.T @ d @ EqA


def grad_f(x):
    return A @ x - b


def hessian_f(x):
    return A


def grad_F(x):
    return lambda t: t * grad_f(x) + grad_Fi(x)


def hessian_F(x):
    return lambda t: t * hessian_f(x) + hessian_Fi(x)


def interior_point_method(x_start):
    cur_x = x_start.copy()
    t = 1.5
    alpha = 1.5
    trajectory = []
    for i in range(50):
        cur_x = cur_x - np.linalg.inv(hessian_F(cur_x)(t)) @ grad_F(cur_x)(t)
        trajectory.append(cur_x.copy())
        t = alpha * t
    return trajectory


interior_point = get_interior_point(EqA, EqB)
print(interior_point)
interior_point_method(interior_point)


[0.5, 0.5, 0.5]


[array([0.49262899, 0.25307125, 0.01351351]),
 array([0.48909026, 0.23616613, 0.02579386]),
 array([0.49981487, 0.18192722, 0.04558183]),
 array([0.51165169, 0.13730625, 0.06510775]),
 array([0.57058282, 0.09771883, 0.05317092]),
 array([0.67609319, 0.05983513, 0.02499042]),
 array([0.72608074, 0.0403517 , 0.02160139]),
 array([0.79760301, 0.02248091, 0.00874722]),
 array([0.82602466, 0.016503  , 0.00861718]),
 array([0.86823022, 0.00858997, 0.00355054]),
 array([0.88797114, 0.0068487 , 0.0034879 ]),
 array([0.91284326, 0.00343939, 0.00154751]),
 array([0.92693944, 0.00288206, 0.00144987]),
 array([9.42098234e-01, 1.44036004e-03, 6.80853265e-04]),
 array([9.51949852e-01, 1.22738533e-03, 6.14440657e-04]),
 array([9.61471766e-01, 6.21864864e-04, 3.01549387e-04]),
 array([9.68233525e-01, 5.28115005e-04, 2.63971330e-04]),
 array([9.74346703e-01, 2.72968939e-04, 1.34093254e-04]),
 array([9.78931914e-01, 2.29118625e-04, 1.14489098e-04]),
 array([9.82913349e-01, 1.20826317e-04, 5.97661765e-05